In [2]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [4]:
import config

In [5]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [6]:
    from IPython.display import IFrame


### Loading data and models

In [7]:
df_hot = pd.read_csv('data/hot_songs_spotify.csv')

In [19]:
df_spotify = pd.read_csv('data/spotify_data_with_clusters.csv')

In [15]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 

    except FileNotFoundError: 
        print("File not found!")

In [16]:
scaler= load(filename="Model/scaler.pickle")

In [17]:
kmeans_model = load(filename="Model/kmeans_7.pickle")

### Functions

In [9]:
def get_hot_recommendation(df_hot, track_id):
    """
    This function picks a random song from the Billboard Hot 100 and returns it in a dataframe with one row.
    It does not pick the same song.
    """
    df_without_title = df_hot.drop(df_hot.loc[df_hot['id']==track_id].index)
    df_recommendation = df_without_title.sample()
    
    return df_recommendation

In [11]:
def predicting_cluster_from_song(track_id, kmeans_model, scaler):
    """
    Extracting the audio features for a song from Spotify API given a track ID.
    Predicting the cluster with my trained KMeans model.
    Returning the predicted cluster
    """
    track_df = pd.DataFrame(sp.audio_features(track_id)[0], index=[0])
    
    track_df_num = track_df.drop(['type', 'key', 'mode', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis = 1)
    
    scaler.fit(track_df_num)
    X_scaled = scaler.transform(track_df_num)
    X_scaled_df = pd.DataFrame(X_scaled, columns = track_df_num.columns)
    
    cluster = kmeans_model.predict(X_scaled_df)
    
    return cluster[0]

In [12]:
def suggested_song_id(df_with_cluster, cluster):
    """
    Taking 
    """
    song_id = df_with_cluster[df_with_cluster['cluster'] == cluster].sample()['id'].iloc[0]
    return song_id

In [13]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

### Program

In [25]:
while True:
    title = input('Enter a song you like')
    results = sp.search(q=f"{title}",limit=5, market='DE')
    song_name = results["tracks"]["items"][0]["name"]
    artist_name = results["tracks"]["items"][0]["artists"][0]["name"]
    track_id = results["tracks"]["items"][0]["id"]
    question = input(f"Do you mean {song_name} from {artist_name}? (y/n)")
    if question == 'y':
        if track_id in list(df_hot['id']):
            df_recommendation = get_hot_recommendation(df_hot, track_id)
            print(f"You will like {df_recommendation['track_title'].iloc[0]} from {df_recommendation['artist'].iloc[0]}!")
            play_song(track_id)
        else:
            cluster = predicting_cluster_from_song(track_id, kmeans_model, scaler)
            song_id = suggested_song_id(df_spotify, cluster)
            print("You will like this song:")
            play_song(song_id)
            break
    if question == 'n':
        print("Please specify your search request or choose another song.")


Enter a song you like ghost
Do you mean Ghost Town from VIZE? (y/n) n


Please specify your search request or choose another song.


Enter a song you like sfd
Do you mean SAD! from XXXTENTACION? (y/n) sdfd
Enter a song you like x
Do you mean XO Tour Llif3 from Lil Uzi Vert? (y/n) y


You will like this song:


In [48]:
play_song(song_id)